In [1]:
# -*- coding: utf-8 -*-
# saver.save()で保存したcheckpointのmeta(NNモデル)と値を読み込み、学習値込みのNNモデルをmodel.pbに保存する

import os, argparse
import tensorflow as tf
from tensorflow.python.framework import graph_util

MODEL_DIR="../model_car_lidar_queue"
FROZEN_MODEL_NAME="car_lidar_queue_20000.pb"
OUTPUT_NODE_NAMES="queue/dequeue_op,neural_network_model/output_y,accuracy/accuracy"

CLEAR_DEVICES=True

def print_graph_operations(graph):
    # print operations
    print "----- operations in graph -----"
    for op in graph.get_operations():
        print op.name,op.outputs 
        
def print_graph_nodes(graph_def):
    # print nodes
    print "----- nodes in graph_def -----"
    for node in graph_def.node:
        print(node)

def freeze_graph():
    global MODEL_DIR
    global CLEAR_DEVICES
    global OUTPUT_NODE_NAMES
    global FROZEN_MODEL_NAME

    # Graphを初期化する(実行エラーで古いGraphが残っている場合に消す)
    tf.reset_default_graph()
    
    checkpoint = tf.train.get_checkpoint_state(MODEL_DIR)
    if checkpoint:
        # checkpointファイルから最後に保存したモデルへのパスを取得する
        last_model = checkpoint.model_checkpoint_path
        print(("load {0}".format(last_model)))

        # pbファイル名を設定する
        # We precise the file fullname of our freezed graph
        absolute_model_dir = "/".join(last_model.split('/')[:-1])
        frozen_model = absolute_model_dir + "/" + FROZEN_MODEL_NAME
        
        # Graphを読み込む
        # We import the meta graph and retrieve a Saver
        saver = tf.train.import_meta_graph(last_model + '.meta', clear_devices=CLEAR_DEVICES)
        # We retrieve the protobuf graph definition
        graph = tf.get_default_graph()
        graph_def = graph.as_graph_def()

        # print operations
        print_graph_operations(graph)

        # print nodes
        #print_graph_nodes(graph_def)

            
    else:
        # checkpointファイルが見つからない
        print("cannot find checkpoint.")
        return


    # We start a session and restore the graph weights
    with tf.Session() as sess:
        # 学習済みモデルの値を読み込む
        saver.restore(sess, last_model)

        # We use a built-in TF helper to export variables to constants
        output_graph_def = graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            graph_def, # The graph_def is used to retrieve the nodes 
            OUTPUT_NODE_NAMES.split(",") # The output node names are used to select the usefull nodes
        )

        # Finally we serialize and dump the output graph to the filesystem
        with tf.gfile.GFile(frozen_model, "wb") as f:
            f.write(output_graph_def.SerializeToString())
            print("%d ops in the final graph." % len(output_graph_def.node))


        # グラフを書き出し
        tf.train.write_graph(output_graph_def, MODEL_DIR,
                             FROZEN_MODEL_NAME, as_text=False)


freeze_graph()



load ../model_car_lidar_queue/model.ckpt
----- operations in graph -----
input/input_data [<tf.Tensor 'input/input_data:0' shape=(?, 3) dtype=float32>]
input/input_target [<tf.Tensor 'input/input_target:0' shape=<unknown> dtype=float32>]
input/batch_size [<tf.Tensor 'input/batch_size:0' shape=<unknown> dtype=int32>]
queue/FIFOQueue [<tf.Tensor 'queue/FIFOQueue:0' shape=() dtype=resource>]
queue/enqueue_op []
queue/dequeue_op [<tf.Tensor 'queue/dequeue_op:0' shape=(?, 3) dtype=float32>, <tf.Tensor 'queue/dequeue_op:1' shape=(?, 4) dtype=float32>]
neural_network_model/random_normal/shape [<tf.Tensor 'neural_network_model/random_normal/shape:0' shape=(2,) dtype=int32>]
neural_network_model/random_normal/mean [<tf.Tensor 'neural_network_model/random_normal/mean:0' shape=() dtype=float32>]
neural_network_model/random_normal/stddev [<tf.Tensor 'neural_network_model/random_normal/stddev:0' shape=() dtype=float32>]
neural_network_model/random_normal/RandomStandardNormal [<tf.Tensor 'neural_net

INFO:tensorflow:Froze 8 variables.
Converted 8 variables to const ops.
38 ops in the final graph.
